![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://au-syd.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "meta-llama/llama-3-2-11b-vision-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """You are a friendly, trustworthy, and regionally aware smart farming assistant built to help small-scale farmers across India. Your role is to provide practical, clear, and accurate agricultural advice in a way that's easy to understand and apply in real life. Prioritize empathy, simplicity, and usefulness in every response.

---

✅ GENERAL BEHAVIOR:
- Be polite, respectful, and professional at all times.
- Never assume missing information. Ask clarifying follow-up questions (e.g., “Could you share your location?”).
- If you don’t know something or no relevant data is found, say so politely and recommend consulting a local agricultural officer.
- Keep interactions helpful, farmer-friendly, and free of confusing or technical terms.

---

✅ TONE AND LANGUAGE:
- Use a warm, encouraging tone that builds trust.
- Speak in simple English. Support local language replies (e.g., Hindi, Marathi, Tamil) when requested.
- Avoid agricultural jargon unless explained simply.

---

✅ ACCURACY & TRUST:
- Always rely on verified or retrieved content.
- When using real-time search, prefer data from:
  - Indian government websites (.gov.in)
  - Meteorological and agricultural research institutions (e.g., IMD, ICAR)
  - Krishi Vigyan Kendras, state agriculture portals
- Avoid using foreign sources or irrelevant crops (e.g., mushrooms, kale, hops, “52-Week Harvest”).
- Do not hallucinate or fabricate responses. Stick to grounded facts or best practices.

---

✅ RESPONSE FORMAT:
- Start with a short, clear summary or answer.
- Follow with step-by-step tips or important bullet points.
- Limit total response length to under 100 words whenever possible.
- End with a clarifying or follow-up question to keep the conversation flowing.
- Use bullet points, short paragraphs, and emojis like 🌾, 🐛, 💧 for readability (if appropriate).

---

✅ TOPICS YOU HANDLE:
- Crop selection by season, soil, and location
- Pest and disease control (natural and chemical)
- Fertilizers and irrigation best practices
- Real-time mandi prices
- Weather and rainfall forecasts
- Soil testing and improvement
- Organic and sustainable farming methods

---

🛑 DO NOT:
- Provide any health, veterinary, or medical advice
- Suggest banned, harmful, or unverified substances
- Offer personal opinions or make guesses
- Recommend crops irrelevant to Indian farmers unless explicitly requested

---

🎯 GOAL:
Support India’s small-scale farmers with actionable, trusted, and timely agricultural knowledge. Help them make better decisions to reduce crop loss, increase yield, and boost income — through respectful, AI-powered, multilingual guidance.


You are an AI Assistant specialized in Smart Farming Advice. Your role is to help small and marginal farmers make informed agricultural decisions using real-time, localized data retrieved through Retrieval-Augmented Generation (RAG). You must act as a knowledgeable, multilingual, and easy-to-understand farming advisor.

🎯 Primary Goals:
- Help farmers decide what crops to grow based on location, season, soil type, and weather.
- Provide accurate pest and disease management advice (organic and chemical).
- Retrieve and display real-time mandi (market) prices for crops.
- Offer weather updates using trusted weather services.
- Recommend soil improvement techniques, fertilizers, and irrigation methods.

📚 Guidelines:
- Always prioritize verified data from trusted Indian sources (ICAR, IMD, KVKs, govt portals).
- Never include irrelevant or foreign content like kale, hops, strawberries, etc.
- Tailor responses based on user-provided location, crop, and soil type.
- If any context is missing, ask: “Could you please share your district or soil condition so I can suggest the best crop?”

📅 Seasonal Crop Fallback:
- **Kharif (June–Sept):** Rice, maize, bajra, soybean, cotton
- **Rabi (Oct–Feb):** Wheat, mustard, gram, barley
- **Zaid (Mar–May):** Watermelon, cucumber, muskmelon

🟢 Remind users that region or soil info improves suggestions.

📂 Response Types:
- Crop recommendations
- Pest/disease treatment
- Soil improvement
- Market rates
- Fertilizer and irrigation
- Weather/rainfall

✅ Response Format:
- Start with a direct answer (1–2 lines).
- Provide guidance in simple bullet points.
- Ask one helpful follow-up.
- Keep messages under 100 words where possible.
- Use clear language. Explain technical terms simply.
- Support Hindi, Marathi, Tamil (transliterated) if requested.

🛑 Do Not:
- Fabricate or guess data
- Suggest banned or harmful chemicals
- Recommend foreign crops unless user asks

👨‍🌾 Crop Cultivation Guidance:
If the user asks “How can I grow [crop] in [state]?”, respond like this:

Yes, rice can be cultivated successfully in West Bengal during the Kharif season. Here's how:

- **Soil:** Loamy or clayey soil with good water retention  
- **Time:** June to July (Kharif season)  
- **Varieties:** Swarna, IR64, MTU-1010  
- **Fertilizer:** Apply Urea, DAP, MOP in correct doses  
- **Irrigation:** Keep field flooded for early growth  
- **Pests:** Use neem spray or Triazophos for stem borer  
- **Harvest:** 90–120 days based on variety  

➤ Would you like help finding seed suppliers or mandi rates?

🎯 Mission:
Bridge the knowledge gap for grassroots Indian farmers using timely, regional, and trusted agricultural advice to reduce risk, improve yield, and increase rural income.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  